In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!unzip -q '/content/drive/My Drive/dataset.zip'

In [0]:
import random
import numpy as np
import glob
import os
from PIL import Image

In [0]:
def generate_overlay(background, foreground, x, y):
    
    h, w = foreground.shape[0], foreground.shape[1]

    foreground_image = foreground[..., :3]
    mask = (foreground[..., 3:]!=0)*1

    background[y:y+h, x:x+w] = (1.0 - mask) * background[y:y+h, x:x+w] + mask * foreground_image

    return background

In [0]:
def generate_mask(background, foreground, x, y):

    h, w = foreground.shape[0], foreground.shape[1]

    foreground_image = (foreground[..., 3]!=0)*255
    mask = (foreground[..., 3]!=0)*1

    background[y:y+h, x:x+w] = (1.0 - mask) * background[y:y+h, x:x+w] + mask * foreground_image

    return background

In [0]:
def generate_xy(foreground, background):
    bg_w = background.shape[1]
    bg_h = background.shape[0]
    fg_w = foreground.shape[1]
    fg_h = foreground.shape[0]
    x_max = bg_w - fg_w
    y_max = bg_h - fg_h
    random.seed(21)
    x = random.sample(range(0,x_max), 40)
    random.seed(42)
    y = random.sample(range(0,y_max), 40)
    return x, y

In [0]:
def foreground_resize(height, foreground):
    width = int(foreground.shape[1] * height / foreground.shape[0])
    # dsize
    dsize = (width, height)
    # resize image
    output = np.array(Image.fromarray(foreground).resize(size=dsize))
    return output

In [0]:
overlay_folder = '/content/data/fg_bg/'
mask_folder = '/content/data/fg_bg_mask/'
background_folder = '/content/dataset/bg/'
foreground_folder = '/content/dataset/fg/'

In [0]:
os.mkdir(overlay_folder)
os.mkdir(mask_folder)

In [0]:
background_paths = glob.glob(background_folder+'*')
foreground_paths = glob.glob(foreground_folder+'*')


for bg_path in background_paths:
    
    #creating folders corresponding to background
    bg_folder_name = (bg_path.split('/')[-1].split('\\')[-1].split('.')[0])+'/'
    os.mkdir(str(overlay_folder+bg_folder_name))
    os.mkdir(str(mask_folder+bg_folder_name))
    
    #reading background image
    background_image = np.array(Image.open(bg_path))
    img_counter = 1
    
    #looping over all the foreground images
    for fg_path in foreground_paths:
        
        #reading foreground image
        foreground_image_temp = np.array(Image.open(fg_path))
        
        #resizing foreground image
        foreground_image = foreground_resize(105, foreground_image_temp.copy())
        
        #generating 40 random positions to place foreground
        x, y =  generate_xy(foreground_image, background_image)
        
        #overlaying 20 foreground
        for i in range(20):
            
            #creating copies to process
            bg_image = background_image.copy()
            fg_image = foreground_image.copy()
            
            #creting black background for mask
            bg_black = np.zeros((224, 224), dtype = "uint8") 
            
            overlaid_image = generate_overlay(bg_image, fg_image, x[i], y[i])
            mask_image = generate_mask(bg_black, fg_image, x[i], y[i])
            
            #saving overlaid and mask images
            Image.fromarray(overlaid_image).save(overlay_folder+bg_folder_name+"img_{0:04}.jpg".format(img_counter),optimize=True,quality=65)
            Image.fromarray(mask_image).save(mask_folder+bg_folder_name+"img_{0:04}.jpg".format(img_counter),optimize=True,quality=65)
            
            #increment the image counter
            img_counter+=1
            
            
        #overlaying 20 flipped foreground
        for i in range(20, 40):
            
            #creating copies to process
            bg_image = background_image.copy()
            fg_image = np.fliplr(foreground_image) #flipping foreground image
            
            #creting black background for mask
            bg_black = np.zeros((224, 224), dtype = "uint8") 
            
            overlaid_image = generate_overlay(bg_image, fg_image, x[i], y[i])
            mask_image = generate_mask(bg_black, fg_image, x[i], y[i])
            
            #saving overlaid and mask images
            Image.fromarray(overlaid_image).save(overlay_folder+bg_folder_name+"img_{0:04}.jpg".format(img_counter),optimize=True,quality=65)
            Image.fromarray(mask_image).save(mask_folder+bg_folder_name+"img_{0:04}.jpg".format(img_counter),optimize=True,quality=65)
            
            #increment the image counter
            img_counter+=1

In [0]:
!zip -r '/content/drive/My Drive/data.zip' 'data'

In [0]:
!unzip -q '/content/drive/My Drive/data.zip'